In [1]:
import os
import sys

sys.path.append('..')

from models import *

import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
le = LabelEncoder()

df = pd.read_hdf("../data_extract/features_dataset_2b.h5")

In [2]:
NUM_PEOPLE = 40
df = df.loc[df['id']<NUM_PEOPLE]

In [3]:
flist = ["mfcc{}".format(i) for i in range(14)]

df_train, df_cross, df_test = get_partitions(df)

obj = MultiGauss(flist)
obj.fit(df_train)

In [4]:
obj.predict_df(df_cross)

             precision    recall  f1-score   support

          0       0.94      1.00      0.97        15
          1       0.90      0.95      0.92        19
          2       1.00      1.00      1.00         9
          3       0.73      0.92      0.81        12
          4       0.68      1.00      0.81        13
          5       0.69      0.60      0.64        15
          6       1.00      0.75      0.86        12
          7       0.88      0.94      0.91        16
          8       1.00      1.00      1.00        11
          9       1.00      1.00      1.00        11
         10       0.94      0.84      0.89        19
         11       1.00      1.00      1.00        12
         12       1.00      0.78      0.88        18
         13       1.00      0.80      0.89        15
         14       0.80      1.00      0.89        12
         15       1.00      1.00      1.00         8
         16       0.91      1.00      0.95        10
         17       1.00      1.00      1.00   